In [104]:
import pandas as pd
import pdfplumber
from pdfplumber.utils.pdfinternals import resolve_and_decode, resolve
from pprint import pprint
import os


### Select relevant data:

In [105]:
'''These are all of the fields which I think are relevant: Operator, time and place, vehicle details, damage report, 
other party details, description, mode, conditions. NOTE: many fields which are positive are marked with a BLANK " ", 
while negatives marked with "None".'''

# Files location
collisions_path = "data/collisions"

all_fields = ['MANufACTuRERS NAME','BuSINESS NAME',
              'DATE Of ACCIDENT','Time of Accident','AM','PM',
              'VEhICLE YEAR','MAkE','MODEL',
              'section 2  accident infoRmation.0','section 2  accident infoRmation.1.0','section 2  accident infoRmation.1.1.0','section 2  accident infoRmation.1.1.1.0','section 2  accident infoRmation.1.1.1.1',
              'Moving', 'Stopped in Traffic', 'Pedestrian', 'Bicyclist', 'undefined', 'Other',
              'NuMBER Of VEhICLES INVOLVED',
              'Unknown','None','minor','Moderate','major',
              'Left Rear 1','Rear Bumper','Right Rear 1','Left Rear 2','Left Rear 3','Right Rear 2','Right Rear 3',
              'Left Rear Passenger 1','Left Rear Passenger 2','Right Rear Passenger 1','Right Rear Passenger 2',
              'Left Rear Passenger 3','Left Rear Passenger 4','Right Rear Passenger 3','Right Rear Passenger 4',
              'Front Driver Side 1','Front Driver Side 2','Front Passenger Side 1','Front Passenger Side 2',
              'Front Driver Side 3','Front Driver Side 4','Front Passenger Side 3','Front Passenger Side 4',
              'Left Front Corner 1','Left Front Corner 2','Right Front Corner 1', 'Right Front Corner 2',
              'Left Front Corner 3','Front Bumper','Right Front Corner 3',
              'Moving_2', 'Stopped in Traffic_2','Pedestrian_2','Bicyclist_2','undefined_2','Other_2',
              'ADDRESS_2.1.0.1','Autonomous Mode','Conventional Mode',
              'WEATHER A 1','WEATHER A 2','WEATHER B 1','WEATHER B 2','WEATHER C 1','WEATHER C 2',
              'WEATHER D 1','WEATHER D 2','WEATHER E 1','WEATHER E 2','WEATHER F 1','WEATHER F 2','WEATHER G 1','WEATHER G 2', 
              'LIGHTING A 1','LIGHTING A 2','LIGHTING B 1','LIGHTING B 2','LIGHTING C 1','LIGHTING C 2',
              'LIGHTING D 1','LIGHTING D 2','LIGHTING E 1','LIGHTING E 2',
              'ROADWAY A 1','ROADWAY A 2','ROADWAY B 1','ROADWAY B 2','ROADWAY C 1', 'ROADWAY C 2','ROADWAY D 1','ROADWAY D 2', 
              'ROAD CONDITIONS A 1','ROAD CONDITIONS A 2','ROAD CONDITIONS B 1','ROAD CONDITIONS B 2', 'ROAD CONDITIONS C 1','ROAD CONDITIONS C 2',
              'ROAD CONDITIONS D 1','ROAD CONDITIONS D 2','ROAD CONDITIONS E 1', 'ROAD CONDITIONS E 2', 'ROAD CONDITIONS F 1', 'ROAD CONDITIONS F 2', 
              'ROAD CONDITIONS G 1', 'ROAD CONDITIONS G 2', 'ROAD CONDITIONS H 1', 'ROAD CONDITIONS H 2',
              'MOVEMENT A 1','MOVEMENT A 2','MOVEMENT  B 1', 'MOVEMENT  B 2','MOVEMENT C 1','MOVEMENT C 2', 'MOVEMENT  D 1','MOVEMENT  D 2', 
              'MOVEMENT  E 1','MOVEMENT  E 2', 'MOVEMENT  F 1', 'MOVEMENT  F 2', 'MOVEMENT  G 1','MOVEMENT  G 2', 'MOVEMENT  H 1','MOVEMENT  H 2', 
              'MOVEMENT  I 1', 'MOVEMENT  I 2', 'MOVEMENT J 1', 'MOVEMENT J 2', 'MOVEMENT  K 1', 'MOVEMENT  K 2', 'MOVEMENT  L 1', 'MOVEMENT  L 2',
              'MOVEMENT  M 1', 'MOVEMENT  M 2','MOVEMENT  N 1', 'MOVEMENT  N 2', 'MOVEMENT  O 1', 'MOVEMENT  O 2',
              'MOVEMENT  P 1', 'MOVEMENT  P 2', 'MOVEMENT  Q 1', 'MOVEMENT  Q 2', 'MOVEMENT  R 1', 'MOVEMENT  R 2',
              'TYPE A 1', 'TYPE A 2', 'TYPE B 1', 'TYPE B 2','TYPE C 1','TYPE C 2','TYPE D 1','TYPE D 2','TYPE E 1','TYPE E 2','TYPE F 1','TYPE F 2','TYPE G 1','TYPE G 2','TYPE H 1','TYPE H 2',
              'OTHER A YES','OTHER A NO','OTHER B','OTHER C','OTHER D','OTHER E','OTHER F','OTHER G',
              'OTHER H YES','OTHER H NO','OTHER I','OTHER J','OTHER K','OTHER L']

conditions_only = ['WEATHER A 1','WEATHER A 2','WEATHER B 1','WEATHER B 2','WEATHER C 1','WEATHER C 2',
              'WEATHER D 1','WEATHER D 2','WEATHER E 1','WEATHER E 2','WEATHER F 1','WEATHER F 2','WEATHER G 1','WEATHER G 2', 
              'LIGHTING A 1','LIGHTING A 2','LIGHTING B 1','LIGHTING B 2','LIGHTING C 1','LIGHTING C 2',
              'LIGHTING D 1','LIGHTING D 2','LIGHTING E 1','LIGHTING E 2',
              'ROADWAY A 1','ROADWAY A 2','ROADWAY B 1','ROADWAY B 2','ROADWAY C 1', 'ROADWAY C 2','ROADWAY D 1','ROADWAY D 2', 
              'ROAD CONDITIONS A 1','ROAD CONDITIONS A 2','ROAD CONDITIONS B 1','ROAD CONDITIONS B 2', 'ROAD CONDITIONS C 1','ROAD CONDITIONS C 2',
              'ROAD CONDITIONS D 1','ROAD CONDITIONS D 2','ROAD CONDITIONS E 1', 'ROAD CONDITIONS E 2', 'ROAD CONDITIONS F 1', 'ROAD CONDITIONS F 2', 
              'ROAD CONDITIONS G 1', 'ROAD CONDITIONS G 2', 'ROAD CONDITIONS H 1', 'ROAD CONDITIONS H 2',
              'MOVEMENT A 1','MOVEMENT A 2','MOVEMENT  B 1', 'MOVEMENT  B 2','MOVEMENT C 1','MOVEMENT C 2', 'MOVEMENT  D 1','MOVEMENT  D 2', 
              'MOVEMENT  E 1','MOVEMENT  E 2', 'MOVEMENT  F 1', 'MOVEMENT  F 2', 'MOVEMENT  G 1','MOVEMENT  G 2', 'MOVEMENT  H 1','MOVEMENT  H 2', 
              'MOVEMENT  I 1', 'MOVEMENT  I 2', 'MOVEMENT J 1', 'MOVEMENT J 2', 'MOVEMENT  K 1', 'MOVEMENT  K 2', 'MOVEMENT  L 1', 'MOVEMENT  L 2',
              'MOVEMENT  M 1', 'MOVEMENT  M 2','MOVEMENT  N 1', 'MOVEMENT  N 2', 'MOVEMENT  O 1', 'MOVEMENT  O 2',
              'MOVEMENT  P 1', 'MOVEMENT  P 2', 'MOVEMENT  Q 1', 'MOVEMENT  Q 2', 'MOVEMENT  R 1', 'MOVEMENT  R 2',
              'TYPE A 1', 'TYPE A 2', 'TYPE B 1', 'TYPE B 2','TYPE C 1','TYPE C 2','TYPE D 1','TYPE D 2','TYPE E 1','TYPE E 2','TYPE F 1','TYPE F 2','TYPE G 1','TYPE G 2','TYPE H 1','TYPE H 2',
              'OTHER A YES''OTHER A NO','OTHER B','OTHER C','OTHER D','OTHER E','OTHER F','OTHER G',
              'OTHER H YES','OTHER H NO','OTHER I','OTHER J','OTHER K','OTHER L']

damage_only = ['Unknown','None','minor','Moderate','major',
              'Left Rear 1','Rear Bumper','Right Rear 1','Left Rear 2','Left Rear 3','Right Rear 2','Right Rear 3',
              'Left Rear Passenger 1','Left Rear Passenger 2','Right Rear Passenger 1','Right Rear Passenger 2',
              'Left Rear Passenger 3','Left Rear Passenger 4','Right Rear Passenger 3','Right Rear Passenger 4',
              'Front Driver Side 1','Front Driver Side 2','Front Passenger Side 1','Front Passenger Side 2',
              'Front Driver Side 3','Front Driver Side 4','Front Passenger Side 3','Front Passenger Side 4',
              'Left Front Corner 1','Left Front Corner 2','Right Front Corner 1', 'Right Front Corner 2',
              'Left Front Corner 3','Front Bumper','Right Front Corner 3',]

geo_only = ['MANufACTuRERS NAME','BuSINESS NAME',
              'DATE Of ACCIDENT','Time of Accident','AM','PM',
              'section 2  accident infoRmation.0','section 2  accident infoRmation.1.0','section 2  accident infoRmation.1.1.0',
              'section 2  accident infoRmation.1.1.1.0','section 2  accident infoRmation.1.1.1.1']




In [106]:
def parse_field_helper(form_data, field, prefix=None):
    """ appends any PDF AcroForm field/value pairs in `field` to provided `form_data` list

        if `field` has child fields, those will be parsed recursively.
    """
    resolved_field = field.resolve()
    field_name = '.'.join(filter(lambda x: x, [prefix, resolve_and_decode(resolved_field.get("T"))]))
    if "Kids" in resolved_field:
        for kid_field in resolved_field["Kids"]:
            parse_field_helper(form_data, kid_field, prefix=field_name)
    if "T" in resolved_field or "TU" in resolved_field:
        # "T" is a field-name, but it's sometimes absent.
        # "TU" is the "alternate field name" and is often more human-readable
        # your PDF may have one, the other, or both.
        alternate_field_name  = resolve_and_decode(resolved_field.get("TU")) if resolved_field.get("TU") else None
        field_value = resolve_and_decode(resolved_field["V"]) if 'V' in resolved_field else None
        
        # Remove non-printable characters and trailing spaces - This affects every Cruise file.
        field_name = ''.join(char for char in field_name if char.isprintable()).strip()
        alternate_field_name = ''.join(char for char in alternate_field_name if char.isprintable()).strip() if alternate_field_name else None
        field_value = ''.join(char for char in field_value if char.isprintable()).strip() if field_value else None
        
        form_data.append([field_name, alternate_field_name, field_value])


# Define filtering criteria 
def find_important_tuples(tuple, search_condition):
    # Check if the first element of the tuple is in the search condition list
    return tuple[0] in search_condition


# Loop over all pdfs and add entries.  Keyword added for interrogating Cruise, specifically
def extract_from_pdf(list_of_pdf_fields, keyword=None):
    # List of collisions
    collisions = []
    # Create an empty dictionary to store data for DataFrame
    counter = 0
    for filename in os.listdir(collisions_path):
        # Check if the current item is a file (not a subdirectory)
        if os.path.isfile(os.path.join(collisions_path, filename)) and (keyword is None or keyword in filename):
            data_dict = {}
            print(f'Extracting:{collisions_path}/{filename}...')


            # Open each pdf
            pdf = pdfplumber.open(f'{collisions_path}/{filename}')
            # initialize fom data list
            form_data = []
            # identify fields
            fields = resolve(pdf.doc.catalog["AcroForm"])["Fields"]

            # For each field, run the pdf parsing function to extract adta and add it to form_data list
            for field in fields:
                parse_field_helper(form_data, field)
                
            # Filter the long list of tuples [all_fields, geo_only, conditions_only, damage_only]
            filtered_list = [tuple for tuple in form_data if find_important_tuples(tuple, list_of_pdf_fields)]

            # Set df sturcture so each pdf is one row - alt_text is column name, value is vlaue]
            # Populate the dictionary with values from filtered_list
            for tuple in filtered_list:
                column_name = tuple[1]
                row_value = tuple[2]
                data_dict[column_name] = row_value

            # Create DataFrame from the dictionary
            collision_report = pd.DataFrame([data_dict])
            collisions.append(collision_report)
            print('Done.')
            counter +=1 

    print(f"Extracted data from {counter} collision reports.")
            
    df = pd.concat(collisions)    
    # Reset the index of the DataFrame
    df.reset_index(drop=True, inplace=True)

    return df

In [107]:
collision_locations = extract_from_pdf(geo_only)
collision_locations.head(20)


Extracting:data/collisions/Apollo-OL316-062923-Redacted.pdf...
Done.
Extracting:data/collisions/Apollo-OL316-090623-Redacted.pdf...
Done.
Extracting:data/collisions/Apollo-OL316-101623-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-021524-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-051123-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-092623-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-092923-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-101323-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-102323-Redacted-1.pdf...
Done.
Extracting:data/collisions/Apple_012624.pdf...
Done.
Extracting:data/collisions/Apple_022123.pdf...
Done.
Extracting:data/collisions/Beep-OL316-082323-Redacted.pdf...
Done.
Extracting:data/collisions/Cruise-OL316-042123-Redacted.pdf...
Done.
Extracting:data/collisions/Cruise-OL316-042223-Redacted.pdf...
Done.
Extracting:data/collisions/Cruise-OL316-050323-Redacted.pdf...
Done.
Ext

,Section 1. Manufacturers information. Enter manufacturer's name,enter BUSINESS NAME,Section 2. Accident information vehicle one enter DATE Of ACCIDENT,enter time of accident,Time of accident. Mark if Ay M.,Mark if P M.,enter address and location of accident.,enter city of accident,enter county of accident.,enter state of accident.,enter zip code of accident.
0,Apollo Autonomous Driving USA LLC,Apollo Autonomous Driving USA LLC,06/29/2023,3:39,None,,Moraga St near 34th Ave,San Francisco,San Francisco,CA,94122
1,Apollo Autonomous Driving USA LLC,Apollo Autonomous Driving USA LLC,9/6/2023,12:25,None,,2037 Rivera Street,San Francisco,San Francisco,CA,94116
2,Apollo Autonomous Driving USA LLC,Apollo Autonomous Driving USA LLC,10/16/2023,12:02,None,,1901 46th Avenue,San Francisco,San Francisco,CA,94116
3,Apple Inc.,Apple Inc.,02/15/2024,12:03,None,,US 101N at Ellis St Exit,Sunnyvale,Santa Clara,CA,94089
4,Apple Inc.,Apple Inc.,05/11/2023,04:55,None,,Our Lady's Way & Great America Pkwy,Santa Clara,Santa Clara,CA,95054
5,Apple Inc.,Apple Inc.,09/26/2023,11:33,,None,I-80 Westbound,Emeryville,Alameda,CA,94608
6,Apple Inc.,Apple Inc.,09/29/2023,2:58,None,,US 101S near Capitol Expressway,San Jose,Santa Clara,CA,95121
7,Apple Inc.,Apple Inc.,10/13/2023,1:52,None,,I-880N near Brokaw Rd Exit,San Jose,Santa Clara,CA,95131
8,Apple Inc.,Apple Inc.,10/23/2023,05:06,None,,Lawrence Expressway and Kifer Road,Santa Clara,Santa Clara,CA,95051
9,Apple Inc.,Apple Inc.,01/26/2024,11:06,,None,I-580E and I-238 Junction,Castro Valley,Alameda,CA,94546


In [108]:
# Rename columns:
rename_dict = {"Section 1. Manufacturers information. Enter manufacturer's name": 'Operator',
               "enter BUSINESS NAME": 'Business',
               "Section 2. Accident information vehicle one enter DATE Of ACCIDENT": "Date_of_Accident",
               "enter time of accident": "Time_of_accident",
               "Time of accident. Mark if Ay M.":"AM",
               "Mark if P M.":"PM",
               "enter address and location of accident.":"address",
               "enter city of accident":"city",
               "enter county of accident.":"county",
               "enter state of accident.":"state",
               "enter zip code of accident.":"zip",
               "enter address and location of accident.":"Location",
               "describe accident details.":"Description",
               "section 5. accident details description. Mark if Autonomous Mode.":"In_autonomy",
               "Mark if Conventional Mode":"Conventional_mode"}

collision_locations.rename(columns=rename_dict, inplace=True)

# Fix date times
collision_locations['Date_of_Accident'] = pd.to_datetime(collision_locations['Date_of_Accident'], errors='coerce')

# Get the initial number of rows
initial_rows = collision_locations.shape[0]

# Drop NaN values in 'Time_of_accident' column with an empty string
collision_locations.dropna(subset=['Time_of_accident'], inplace=True)

# Calculate the number of rows dropped
rows_dropped = initial_rows - collision_locations.shape[0]
print(f"Number of rows dropped: {rows_dropped}")

# Convert 'Time_of_accident' column to string type
collision_locations['Time_of_accident'] = collision_locations['Time_of_accident'].astype(str)


# Update time format based on AM and PM columns
for index, row in collision_locations.iterrows():
    time_str = row['Time_of_accident']
    if row['AM'] == ' ':
        # If AM is checked, assume it's AM
        time_str += ' AM'
    elif row['PM'] == ' ':
        # If PM is checked, assume it's PM
        time_str += ' PM'
    else:
        # Assume it's PM
        time_str += ' PM'

    # Update the time string
    collision_locations.at[index, 'Time_of_accident'] = time_str

# Fix Time column
collision_locations['Time_of_accident'] = collision_locations['Time_of_accident'].str.strip()
# Drop Am/PM
collision_locations.drop(columns=['AM', 'PM'], inplace=True)
# Convert back to datetime
collision_locations['Time_of_accident'] = pd.to_datetime(collision_locations['Time_of_accident'], format='%I:%M %p')
# Convert 'Time_of_accident' column to 24-hr HH:MM
collision_locations['Time_of_accident'] = collision_locations['Time_of_accident'].dt.strftime('%H:%M')







Number of rows dropped: 0


In [109]:
collision_locations.head(20)

,Operator,Business,Date_of_Accident,Time_of_accident,Location,city,county,state,zip
0,Apollo Autonomous Driving USA LLC,Apollo Autonomous Driving USA LLC,2023-06-29,15:39,Moraga St near 34th Ave,San Francisco,San Francisco,CA,94122
1,Apollo Autonomous Driving USA LLC,Apollo Autonomous Driving USA LLC,2023-09-06,12:25,2037 Rivera Street,San Francisco,San Francisco,CA,94116
2,Apollo Autonomous Driving USA LLC,Apollo Autonomous Driving USA LLC,2023-10-16,12:02,1901 46th Avenue,San Francisco,San Francisco,CA,94116
3,Apple Inc.,Apple Inc.,2024-02-15,12:03,US 101N at Ellis St Exit,Sunnyvale,Santa Clara,CA,94089
4,Apple Inc.,Apple Inc.,2023-05-11,16:55,Our Lady's Way & Great America Pkwy,Santa Clara,Santa Clara,CA,95054
5,Apple Inc.,Apple Inc.,2023-09-26,23:33,I-80 Westbound,Emeryville,Alameda,CA,94608
6,Apple Inc.,Apple Inc.,2023-09-29,14:58,US 101S near Capitol Expressway,San Jose,Santa Clara,CA,95121
7,Apple Inc.,Apple Inc.,2023-10-13,13:52,I-880N near Brokaw Rd Exit,San Jose,Santa Clara,CA,95131
8,Apple Inc.,Apple Inc.,2023-10-23,17:06,Lawrence Expressway and Kifer Road,Santa Clara,Santa Clara,CA,95051
9,Apple Inc.,Apple Inc.,2024-01-26,23:06,I-580E and I-238 Junction,Castro Valley,Alameda,CA,94546


In [110]:
# export as csv
# Specify the full path where you want to save the CSV file
csv_file_path = 'data/collisions/dataframes/locations/crash_locations.csv'

# Save DataFrame to CSV
collision_locations.to_csv(csv_file_path, index=False)


In [111]:
# TODO: Need to change " "s into True and "None" into False - found different workadround for AM/PM
# TODO: Change the conditions column names (light, weather, etc) into the alt_filed name (tuple[1]) for human readability. - NOTE: partially solved by using Alt_text instead of text. Still should be cleaned


In [133]:
# TODO: @Kyle - create descriptions dataframe. Save and read into main for wordcloud
description_cols = ["section 2  accident infoRmation.0", "Autonomous Mode", "Conventional Mode",'ADDRESS_2.1.0.1']
descriptions_df = extract_from_pdf(description_cols)


Extracting:data/collisions/Apollo-OL316-062923-Redacted.pdf...
Done.
Extracting:data/collisions/Apollo-OL316-090623-Redacted.pdf...
Done.
Extracting:data/collisions/Apollo-OL316-101623-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-021524-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-051123-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-092623-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-092923-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-101323-Redacted.pdf...
Done.
Extracting:data/collisions/Apple-OL316-102323-Redacted-1.pdf...
Done.
Extracting:data/collisions/Apple_012624.pdf...
Done.
Extracting:data/collisions/Apple_022123.pdf...
Done.
Extracting:data/collisions/Beep-OL316-082323-Redacted.pdf...
Done.
Extracting:data/collisions/Cruise-OL316-042123-Redacted.pdf...
Done.
Extracting:data/collisions/Cruise-OL316-042223-Redacted.pdf...
Done.
Extracting:data/collisions/Cruise-OL316-050323-Redacted.pdf...
Done.
Ext

In [134]:
descriptions_df.rename(columns=rename_dict, inplace=True)
# Replace whitespace with "Yes" and None with "No" after stripping any leading or trailing whitespace
descriptions_df["In_autonomy"] = descriptions_df["In_autonomy"].str.strip().replace({"": True, None: False})
descriptions_df["Conventional_mode"] = descriptions_df["Conventional_mode"].str.strip().replace({"": 1, None: 0})

descriptions_df['Conventional_mode'] = descriptions_df['Conventional_mode'].astype(bool)


descriptions_df.to_csv('data/collisions/dataframes/descriptions/descriptions.csv')
descriptions_df.head()

C:\Users\outdo\AppData\Local\Temp\ipykernel_6288\2937102919.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  descriptions_df["In_autonomy"] = descriptions_df["In_autonomy"].str.strip().replace({"": True, None: False})


,Location,Description,In_autonomy,Conventional_mode
0,Moraga St near 34th Ave,"On June 29, 2023 at 3:39 PM PDT, an Apollo Aut...",True,False
1,2037 Rivera Street,An Apollo autonomous vehicle (“AV”) was travel...,False,True
2,1901 46th Avenue,An Apollo autonomous vehicle (“AV”) was travel...,False,True
3,US 101N at Ellis St Exit,"On February 15th, a test vehicle operating wit...",False,True
4,Our Lady's Way & Great America Pkwy,"On May 11th, a test vehicle operating in manua...",False,True


In [114]:
# @JackP TODO: Collision heatmap and severity

In [115]:
# TODO: Geo scatter a) color by year, b) color by mfg

In [116]:
# TODO: Histogram of Day of year of collisions (on main)
# TODO: Histogram of Tim of day of collisions (on main)
# TODO: Collisions & disengagements grouped bar by mfg (on main)
